In [114]:
from astropy.io import fits
from astropy.table import Table, Column, hstack
from astropy.modeling import models, fitting
import pdb
import numpy as np
import rafias_lib as rl
import glob
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [45]:
def bkg_by_med(xcenter, ycenter, raw_flux, r, r_in, r_out, hdu_filenames, red_files = False, rf_slope = False):
    bkg_list = []
    res_flux = []
    if red_files == False:
        for i, (hdus, raw) in enumerate(zip(hdu_filenames, raw_flux)):
            hdu = fits.open(hdus)
            image = hdu[0].data
            image2d = image[0,:,:]
            y, x = np.mgrid[:image2d.shape[0], :image2d.shape[1]]
            bkg_pts = np.where((((x - xcenter[i])**2 + (y - ycenter[i])**2) > (r_in)**2) & 
                               (((x - xcenter[i])**2 + (y -ycenter[i])**2) < (r_out)**2))
            bkg_med = np.nanmedian(image2d[bkg_pts])
            bkg = bkg_med*(np.pi*(r**2))
            bkg_list.append(bkg)
            res = raw - bkg
            res_flux.append(res)
#             pdb.set_trace()
        return bkg_list, res_flux
    else:
        if rf_slope == False:
            for i, (hdus, raw) in enumerate(zip(hdu_filenames, raw_flux)):
                hdu = fits.open(hdus)
                image = hdu[0].data
                header = hdu[0].header
                image2d = (image[-1] - image[0])/((header['NGROUP']-1)*header['TGROUP'])
                y, x = np.mgrid[:image2d.shape[0], :image2d.shape[1]]
                bkg_pts = np.where((((x - xcenter[i])**2 + (y - ycenter[i])**2) > (r_in)**2) & 
                                   (((x - xcenter[i])**2 + (y -ycenter[i])**2) < (r_out)**2))
                bkg_med = np.nanmedian(image2d[bkg_pts])
                bkg = bkg_med*(np.pi*(r**2))
                bkg_list.append(bkg)
                res = raw - bkg
                res_flux.append(res)
            return bkg_list, res_flux
        else:
            for i, (hdus, raw) in enumerate(zip(hdu_filenames, raw_flux)):
                hdu = fits.open(hdus)
                image = hdu[0].data
                header = hdu[0].header
                image2d = image[-1]/(header['NGROUP']*header['TGROUP'])
                y, x = np.mgrid[:image2d.shape[0], :image2d.shape[1]]
                bkg_pts = np.where((((x - xcenter[i])**2 + (y - ycenter[i])**2) > (r_in)**2) & 
                                   (((x - xcenter[i])**2 + (y -ycenter[i])**2) < (r_out)**2))
                bkg_med = np.nanmedian(image2d[bkg_pts])
                bkg = bkg_med*(np.pi*(r**2))
                bkg_list.append(bkg)
                res = raw - bkg
                res_flux.append(res)
            return bkg_list, res_flux

In [60]:
def std_by_med(a1_center, b4_center, r, r_in, r_out, a1_hdu, b4_hdu, rf, rfs):
    a1_raw = rl.time_series(a1_center[0], r, a1_hdu)['Flux']
    b4_raw = rl.time_series(b4_center[0], r, b4_hdu)['Flux']
    a11_res = np.array(bkg_by_med(a1_center[1], a1_center[2], a1_raw, r, r_in, r_out, 
                                  a1_hdu, red_files = rf, rf_slope = rfs)[1])
    b41_res = np.array(bkg_by_med(b4_center[1], b4_center[2], b4_raw, r, r_in, r_out, 
                                  b4_hdu, red_files = rf, rf_slope = rfs)[1])
    av_res1 = (a11_res + b41_res[:306])/2
    std = np.std(av_res1/np.median(av_res1))
    return std

## Test 1 (WLP8SUB)

In [8]:
a11_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8SUB-6012134600_1_481_SE_2016-01-12T16h42m53/*.slp.fits')
len(a11_files)

306

In [9]:
b41_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8SUB-6012134957_1_489_SE_2016-01-12T16h43m29/*.slp.fits')
len(b41_files)

307

In [19]:
a11_centers = rl.gen_center_g2d(166,160,5,400,2,2,0, a11_files)
a11_centers

([(Parameter('x_mean', value=165.957862014),
   Parameter('y_mean', value=160.086898746)),
  (Parameter('x_mean', value=165.95454923),
   Parameter('y_mean', value=160.087290893)),
  (Parameter('x_mean', value=165.971602003),
   Parameter('y_mean', value=160.089622193)),
  (Parameter('x_mean', value=165.959523516),
   Parameter('y_mean', value=160.108219701)),
  (Parameter('x_mean', value=166.011418093),
   Parameter('y_mean', value=160.121689352)),
  (Parameter('x_mean', value=165.971081516),
   Parameter('y_mean', value=160.125357794)),
  (Parameter('x_mean', value=165.938381185),
   Parameter('y_mean', value=160.132461756)),
  (Parameter('x_mean', value=165.915041144),
   Parameter('y_mean', value=160.120045428)),
  (Parameter('x_mean', value=165.981993937),
   Parameter('y_mean', value=160.124199847)),
  (Parameter('x_mean', value=165.969651467),
   Parameter('y_mean', value=160.107808149)),
  (Parameter('x_mean', value=165.982537641),
   Parameter('y_mean', value=160.108742028)),


In [20]:
b41_centers = rl.gen_center_g2d(162,156,5,500,2,2,0, b41_files)
b41_centers

([(Parameter('x_mean', value=159.154083778),
   Parameter('y_mean', value=156.593990113)),
  (Parameter('x_mean', value=159.149132141),
   Parameter('y_mean', value=156.583788553)),
  (Parameter('x_mean', value=159.173824065),
   Parameter('y_mean', value=156.590719663)),
  (Parameter('x_mean', value=159.15173308),
   Parameter('y_mean', value=156.605461423)),
  (Parameter('x_mean', value=159.202971001),
   Parameter('y_mean', value=156.626039547)),
  (Parameter('x_mean', value=159.165093993),
   Parameter('y_mean', value=156.620309476)),
  (Parameter('x_mean', value=159.132477782),
   Parameter('y_mean', value=156.623550823)),
  (Parameter('x_mean', value=159.109227828),
   Parameter('y_mean', value=156.628989799)),
  (Parameter('x_mean', value=159.17167704),
   Parameter('y_mean', value=156.617309121)),
  (Parameter('x_mean', value=159.167540166),
   Parameter('y_mean', value=156.611481065)),
  (Parameter('x_mean', value=159.17838075),
   Parameter('y_mean', value=156.607335825)),
  

In [63]:
med_std1 = std_by_med(a11_centers, b41_centers, 90, 100, 110, a11_files, b41_files, False, False)
med_std1

0.0015800867761350315

In [65]:
data1 = rl.average_residual_flux(a11_centers[0], b41_centers[0], 90, 100, 110, a11_files, b41_files)
aper_std1 = np.std(data1['a1_b4_res_flux']/np.median(data1['a1_b4_res_flux']))
aper_std1

0.0015837643041155679

## Test 2 (WLP8SUB640)

In [66]:
a12_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8SUB640-6012195832_1_481_SE_2016-01-12T20h57m57/*.slp.fits')
len(a12_files)

60

In [67]:
b42_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8SUB640-6012200140_1_489_SE_2016-01-12T20h58m27/*.slp.fits')
len(b42_files)

60

In [72]:
a12_centers = rl.gen_center_g2d(326,320,5,400,2,2,0, a12_files)
a12_centers

([(Parameter('x_mean', value=326.03857685),
   Parameter('y_mean', value=319.793954251)),
  (Parameter('x_mean', value=326.060386731),
   Parameter('y_mean', value=319.827751241)),
  (Parameter('x_mean', value=326.023288539),
   Parameter('y_mean', value=319.804793373)),
  (Parameter('x_mean', value=325.998574504),
   Parameter('y_mean', value=319.798378001)),
  (Parameter('x_mean', value=325.970356655),
   Parameter('y_mean', value=319.803212806)),
  (Parameter('x_mean', value=326.003435332),
   Parameter('y_mean', value=319.811558185)),
  (Parameter('x_mean', value=326.062244814),
   Parameter('y_mean', value=319.840337212)),
  (Parameter('x_mean', value=326.095922221),
   Parameter('y_mean', value=319.84549006)),
  (Parameter('x_mean', value=326.039505305),
   Parameter('y_mean', value=319.835726359)),
  (Parameter('x_mean', value=326.01420748),
   Parameter('y_mean', value=319.849799878)),
  (Parameter('x_mean', value=325.975102671),
   Parameter('y_mean', value=319.829034599)),
  

In [73]:
b42_centers = rl.gen_center_g2d(319,316,5,500,2,2,0, b42_files)
b42_centers

([(Parameter('x_mean', value=319.164777393),
   Parameter('y_mean', value=316.389669782)),
  (Parameter('x_mean', value=319.18841059),
   Parameter('y_mean', value=316.409601551)),
  (Parameter('x_mean', value=319.14416023),
   Parameter('y_mean', value=316.386046104)),
  (Parameter('x_mean', value=319.117608583),
   Parameter('y_mean', value=316.375749558)),
  (Parameter('x_mean', value=319.09821857),
   Parameter('y_mean', value=316.376599081)),
  (Parameter('x_mean', value=319.130722424),
   Parameter('y_mean', value=316.38885253)),
  (Parameter('x_mean', value=319.185116529),
   Parameter('y_mean', value=316.425946596)),
  (Parameter('x_mean', value=319.212951297),
   Parameter('y_mean', value=316.434755612)),
  (Parameter('x_mean', value=319.154522083),
   Parameter('y_mean', value=316.415958021)),
  (Parameter('x_mean', value=319.132166132),
   Parameter('y_mean', value=316.431570351)),
  (Parameter('x_mean', value=319.096324585),
   Parameter('y_mean', value=316.411325002)),
  (

In [74]:
med_std2 = std_by_med(a12_centers, b42_centers, 90, 100, 110, a12_files, b42_files, False, False)
med_std2

0.0023328257547695209

In [75]:
data2 = rl.average_residual_flux(a12_centers[0], b42_centers[0], 90, 100, 110, a12_files, b42_files)
aper_std2 = np.std(data2['a1_b4_res_flux']/np.median(data2['a1_b4_res_flux']))
aper_std2

0.0023432935795392709

## Test 3 (Full 1)

In [81]:
a13_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL1-6012051326_1_481_SE_2016-01-12T08h55m24/*.red.fits')
len(a13_files)

225

In [82]:
b43_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL1-6012051528_1_489_SE_2016-01-12T08h30m07/*.red.fits')
len(b43_files)

225

In [83]:
a13_centers = rl.gen_center_g2d(1405,1036,5,400,4,4,0, a13_files, red_files = True, rf_slope = True)
a13_centers

([(Parameter('x_mean', value=1404.98079062),
   Parameter('y_mean', value=1037.5570441)),
  (Parameter('x_mean', value=1404.99702526),
   Parameter('y_mean', value=1037.58592583)),
  (Parameter('x_mean', value=1404.92039484),
   Parameter('y_mean', value=1037.59495075)),
  (Parameter('x_mean', value=1404.941185),
   Parameter('y_mean', value=1037.5920752)),
  (Parameter('x_mean', value=1404.9555604),
   Parameter('y_mean', value=1037.59670267)),
  (Parameter('x_mean', value=1404.9110232),
   Parameter('y_mean', value=1037.57501656)),
  (Parameter('x_mean', value=1404.93330953),
   Parameter('y_mean', value=1037.54401393)),
  (Parameter('x_mean', value=1404.91267724),
   Parameter('y_mean', value=1037.52932399)),
  (Parameter('x_mean', value=1404.93614084),
   Parameter('y_mean', value=1037.55573259)),
  (Parameter('x_mean', value=1405.02847174),
   Parameter('y_mean', value=1037.57187621)),
  (Parameter('x_mean', value=1405.02050737),
   Parameter('y_mean', value=1037.57466863)),
  (Pa

In [84]:
b43_centers = rl.gen_center_g2d(828,821,5,600,4,4,0, b43_files, red_files = True, rf_slope = True)
b43_centers

([(Parameter('x_mean', value=828.168542886),
   Parameter('y_mean', value=821.09568236)),
  (Parameter('x_mean', value=828.168904571),
   Parameter('y_mean', value=821.119432004)),
  (Parameter('x_mean', value=828.092124218),
   Parameter('y_mean', value=821.127626685)),
  (Parameter('x_mean', value=828.111515959),
   Parameter('y_mean', value=821.128698986)),
  (Parameter('x_mean', value=828.127178215),
   Parameter('y_mean', value=821.140364596)),
  (Parameter('x_mean', value=828.080258845),
   Parameter('y_mean', value=821.118377234)),
  (Parameter('x_mean', value=828.102954904),
   Parameter('y_mean', value=821.083480555)),
  (Parameter('x_mean', value=828.085821375),
   Parameter('y_mean', value=821.060850223)),
  (Parameter('x_mean', value=828.099246141),
   Parameter('y_mean', value=821.074356072)),
  (Parameter('x_mean', value=828.198583265),
   Parameter('y_mean', value=821.105579045)),
  (Parameter('x_mean', value=828.190077731),
   Parameter('y_mean', value=821.114891004)),


In [87]:
med_std3 = std_by_med(a13_centers, b43_centers, 50, 60, 70, a13_files, b43_files, True, True)
med_std3

0.0024462414797297898

In [88]:
data3 = rl.average_residual_flux(a13_centers[0], b43_centers[0], 50, 60, 70, 
                                 a13_files, b43_files, red_files = True, rf_slope = True)
aper_std3 = np.std(data3['a1_b4_res_flux']/np.median(data3['a1_b4_res_flux']))
aper_std3

0.001596058822868295

## Test 4 (Full 2)

In [89]:
a14_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL2-6012072417_1_481_SE_2016-01-12T10h40m58/*.red.fits')
len(a14_files)

225

In [90]:
b44_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL2-6012072532_1_489_SE_2016-01-12T11h09m06/*.red.fits')
len(b44_files)

225

In [91]:
a14_centers = rl.gen_center_g2d(1405,1036,5,400,4,4,0, a14_files, red_files = True, rf_slope = True)
a14_centers

([(Parameter('x_mean', value=1404.87914265),
   Parameter('y_mean', value=1037.57686126)),
  (Parameter('x_mean', value=1404.90460458),
   Parameter('y_mean', value=1037.58932817)),
  (Parameter('x_mean', value=1404.97554574),
   Parameter('y_mean', value=1037.60841607)),
  (Parameter('x_mean', value=1405.02631003),
   Parameter('y_mean', value=1037.60962109)),
  (Parameter('x_mean', value=1405.03885904),
   Parameter('y_mean', value=1037.63115324)),
  (Parameter('x_mean', value=1404.96907717),
   Parameter('y_mean', value=1037.6045562)),
  (Parameter('x_mean', value=1404.96869262),
   Parameter('y_mean', value=1037.63292155)),
  (Parameter('x_mean', value=1404.9685918),
   Parameter('y_mean', value=1037.63666341)),
  (Parameter('x_mean', value=1404.94305521),
   Parameter('y_mean', value=1037.64622332)),
  (Parameter('x_mean', value=1404.88063111),
   Parameter('y_mean', value=1037.63389067)),
  (Parameter('x_mean', value=1404.90029896),
   Parameter('y_mean', value=1037.65367491)),
 

In [92]:
b44_centers = rl.gen_center_g2d(828,821,5,600,4,4,0, b44_files, red_files = True, rf_slope = True)
b44_centers

([(Parameter('x_mean', value=828.075301527),
   Parameter('y_mean', value=821.133148523)),
  (Parameter('x_mean', value=828.083397611),
   Parameter('y_mean', value=821.124447039)),
  (Parameter('x_mean', value=828.153212566),
   Parameter('y_mean', value=821.126801995)),
  (Parameter('x_mean', value=828.19315446),
   Parameter('y_mean', value=821.143698839)),
  (Parameter('x_mean', value=828.203033486),
   Parameter('y_mean', value=821.159725822)),
  (Parameter('x_mean', value=828.148351981),
   Parameter('y_mean', value=821.13126987)),
  (Parameter('x_mean', value=828.137394197),
   Parameter('y_mean', value=821.16096634)),
  (Parameter('x_mean', value=828.138791991),
   Parameter('y_mean', value=821.167369087)),
  (Parameter('x_mean', value=828.120883722),
   Parameter('y_mean', value=821.180621386)),
  (Parameter('x_mean', value=828.058445836),
   Parameter('y_mean', value=821.176649398)),
  (Parameter('x_mean', value=828.070412262),
   Parameter('y_mean', value=821.17885587)),
  (

In [93]:
med_std4 = std_by_med(a14_centers, b44_centers, 50, 60, 70, a14_files, b44_files, True, True)
med_std4

0.0024179216214801329

In [94]:
data4 = rl.average_residual_flux(a14_centers[0], b44_centers[0], 50, 60, 70, 
                                 a14_files, b44_files, red_files = True, rf_slope = True)
aper_std4 = np.std(data4['a1_b4_res_flux']/np.median(data4['a1_b4_res_flux']))
aper_std4

0.0018589239105399643

## Test 5 (Full 3)

In [95]:
a15_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL3-6012093326_1_481_SE_2016-01-12T12h47m44/*.red.fits')
len(a15_files)

225

In [96]:
b45_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL3-6012093432_1_489_SE_2016-01-12T12h52m21/*.red.fits')
len(b45_files)

225

In [97]:
a15_centers = rl.gen_center_g2d(1405,1036,5,400,4,4,0, a15_files, red_files = True, rf_slope = True)
a15_centers

([(Parameter('x_mean', value=1404.84279597),
   Parameter('y_mean', value=1037.76741901)),
  (Parameter('x_mean', value=1404.85252393),
   Parameter('y_mean', value=1037.78500319)),
  (Parameter('x_mean', value=1404.86943349),
   Parameter('y_mean', value=1037.75194337)),
  (Parameter('x_mean', value=1404.87205576),
   Parameter('y_mean', value=1037.74301734)),
  (Parameter('x_mean', value=1404.85807928),
   Parameter('y_mean', value=1037.75661916)),
  (Parameter('x_mean', value=1404.91685186),
   Parameter('y_mean', value=1037.7478553)),
  (Parameter('x_mean', value=1404.9925886),
   Parameter('y_mean', value=1037.77830163)),
  (Parameter('x_mean', value=1404.95588356),
   Parameter('y_mean', value=1037.76147051)),
  (Parameter('x_mean', value=1404.88671842),
   Parameter('y_mean', value=1037.75878329)),
  (Parameter('x_mean', value=1404.91247854),
   Parameter('y_mean', value=1037.72888174)),
  (Parameter('x_mean', value=1404.95175367),
   Parameter('y_mean', value=1037.75460297)),
 

In [115]:
b45_centers = rl.gen_center_g2d(828,821,5,600,4,4,0, b45_files, red_files = True, rf_slope = True)
b45_centers

([(Parameter('x_mean', value=828.023297782),
   Parameter('y_mean', value=821.344666839)),
  (Parameter('x_mean', value=828.021512423),
   Parameter('y_mean', value=821.321613568)),
  (Parameter('x_mean', value=828.032888233),
   Parameter('y_mean', value=821.284591418)),
  (Parameter('x_mean', value=828.040422756),
   Parameter('y_mean', value=821.273013773)),
  (Parameter('x_mean', value=828.023904632),
   Parameter('y_mean', value=821.264952297)),
  (Parameter('x_mean', value=828.075680593),
   Parameter('y_mean', value=821.277334968)),
  (Parameter('x_mean', value=828.15837957),
   Parameter('y_mean', value=821.316934822)),
  (Parameter('x_mean', value=828.121844311),
   Parameter('y_mean', value=821.300255132)),
  (Parameter('x_mean', value=828.041336502),
   Parameter('y_mean', value=821.293172817)),
  (Parameter('x_mean', value=828.065810582),
   Parameter('y_mean', value=821.264969934)),
  (Parameter('x_mean', value=828.115431796),
   Parameter('y_mean', value=821.288928281)),


In [116]:
med_std5 = std_by_med(a15_centers, b45_centers, 50, 60, 70, a15_files, b45_files, True, True)
med_std5

0.0025451652979003371

In [117]:
data5 = rl.average_residual_flux(a15_centers[0], b45_centers[0], 50, 60, 70, 
                                 a15_files, b45_files, red_files = True, rf_slope = True)
aper_std5 = np.std(data5['a1_b4_res_flux']/np.median(data5['a1_b4_res_flux']))
aper_std5

0.0020175163568834195

## Test 6 (Full 4)

In [118]:
a16_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL4-6012114105_1_481_SE_2016-01-12T14h25m53/*.red.fits')
len(a16_files)

225

In [119]:
b46_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL4-6012114212_1_489_SE_2016-01-12T14h58m10/*.red.fits')
len(b46_files)

225

In [120]:
a16_centers = rl.gen_center_g2d(1405,1036,5,400,4,4,0, a16_files, red_files = True, rf_slope = True)
a16_centers

([(Parameter('x_mean', value=1404.77748436),
   Parameter('y_mean', value=1037.92078354)),
  (Parameter('x_mean', value=1404.7772922),
   Parameter('y_mean', value=1037.93684867)),
  (Parameter('x_mean', value=1404.79696693),
   Parameter('y_mean', value=1037.91797353)),
  (Parameter('x_mean', value=1404.8176673),
   Parameter('y_mean', value=1037.92055993)),
  (Parameter('x_mean', value=1404.84171893),
   Parameter('y_mean', value=1037.93291654)),
  (Parameter('x_mean', value=1404.85188715),
   Parameter('y_mean', value=1037.92280528)),
  (Parameter('x_mean', value=1404.8942564),
   Parameter('y_mean', value=1037.94995958)),
  (Parameter('x_mean', value=1404.89986876),
   Parameter('y_mean', value=1037.96862844)),
  (Parameter('x_mean', value=1404.92627268),
   Parameter('y_mean', value=1037.97140211)),
  (Parameter('x_mean', value=1404.95041233),
   Parameter('y_mean', value=1037.97068029)),
  (Parameter('x_mean', value=1404.99153342),
   Parameter('y_mean', value=1037.99010318)),
  

In [121]:
b46_centers = rl.gen_center_g2d(828,821,5,600,4,4,0, b46_files, red_files = True, rf_slope = True)
b46_centers

([(Parameter('x_mean', value=827.930729641),
   Parameter('y_mean', value=821.487252629)),
  (Parameter('x_mean', value=827.915167154),
   Parameter('y_mean', value=821.461169898)),
  (Parameter('x_mean', value=827.933113192),
   Parameter('y_mean', value=821.451490085)),
  (Parameter('x_mean', value=827.946821095),
   Parameter('y_mean', value=821.466246507)),
  (Parameter('x_mean', value=827.973759029),
   Parameter('y_mean', value=821.473160791)),
  (Parameter('x_mean', value=827.98814891),
   Parameter('y_mean', value=821.46634)),
  (Parameter('x_mean', value=828.016969851),
   Parameter('y_mean', value=821.47767851)),
  (Parameter('x_mean', value=828.040616346),
   Parameter('y_mean', value=821.507841892)),
  (Parameter('x_mean', value=828.066342664),
   Parameter('y_mean', value=821.501717101)),
  (Parameter('x_mean', value=828.076415825),
   Parameter('y_mean', value=821.518009987)),
  (Parameter('x_mean', value=828.122313238),
   Parameter('y_mean', value=821.525672402)),
  (Pa

In [122]:
med_std6 = std_by_med(a16_centers, b46_centers, 50, 60, 70, a16_files, b46_files, True, True)
med_std6

0.0024772025481770354

In [123]:
data6 = rl.average_residual_flux(a16_centers[0], b46_centers[0], 50, 60, 70, 
                                 a16_files, b46_files, red_files = True, rf_slope = True)
aper_std6 = np.std(data6['a1_b4_res_flux']/np.median(data6['a1_b4_res_flux']))
aper_std6

0.0013655378082505561

## Test 7 (Full 5)

In [124]:
a17_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL5-6012184421_1_481_SE_2016-01-12T20h26m57/*.red.fits')
len(a17_files)

110

In [125]:
b47_files = glob.glob('/data1/tso_analysis/all_tso_cv3/raw_separated_MMM'
                              '/NRCN821WLP8FULL5-6012185018_1_489_SE_2016-01-12T20h20m15/*.red.fits')
len(b47_files)

110

In [126]:
a17_centers = rl.gen_center_g2d(1405,1036,5,400,4,4,0, a17_files, red_files = True, rf_slope = True)
a17_centers

([(Parameter('x_mean', value=1405.02123644),
   Parameter('y_mean', value=1037.79528079)),
  (Parameter('x_mean', value=1405.12698688),
   Parameter('y_mean', value=1037.82591034)),
  (Parameter('x_mean', value=1405.07357667),
   Parameter('y_mean', value=1037.83678593)),
  (Parameter('x_mean', value=1405.09988486),
   Parameter('y_mean', value=1037.85706581)),
  (Parameter('x_mean', value=1405.13978244),
   Parameter('y_mean', value=1037.8370634)),
  (Parameter('x_mean', value=1405.10710575),
   Parameter('y_mean', value=1037.86912045)),
  (Parameter('x_mean', value=1405.10393093),
   Parameter('y_mean', value=1037.88808566)),
  (Parameter('x_mean', value=1405.04148143),
   Parameter('y_mean', value=1037.87988345)),
  (Parameter('x_mean', value=1405.04684354),
   Parameter('y_mean', value=1037.86707624)),
  (Parameter('x_mean', value=1405.05645375),
   Parameter('y_mean', value=1037.8670492)),
  (Parameter('x_mean', value=1405.02181668),
   Parameter('y_mean', value=1037.8623938)),
  

In [127]:
b47_centers = rl.gen_center_g2d(828,821,5,600,4,4,0, b47_files, red_files = True, rf_slope = True)
b47_centers

([(Parameter('x_mean', value=828.143864113),
   Parameter('y_mean', value=821.425575547)),
  (Parameter('x_mean', value=828.212676464),
   Parameter('y_mean', value=821.454743154)),
  (Parameter('x_mean', value=828.176415657),
   Parameter('y_mean', value=821.464160088)),
  (Parameter('x_mean', value=828.201561919),
   Parameter('y_mean', value=821.476303091)),
  (Parameter('x_mean', value=828.200842623),
   Parameter('y_mean', value=821.440373129)),
  (Parameter('x_mean', value=828.186153172),
   Parameter('y_mean', value=821.49253253)),
  (Parameter('x_mean', value=828.184602358),
   Parameter('y_mean', value=821.49605933)),
  (Parameter('x_mean', value=828.13370722),
   Parameter('y_mean', value=821.490340687)),
  (Parameter('x_mean', value=828.135783168),
   Parameter('y_mean', value=821.466713986)),
  (Parameter('x_mean', value=828.149975107),
   Parameter('y_mean', value=821.482447181)),
  (Parameter('x_mean', value=828.10338392),
   Parameter('y_mean', value=821.468310766)),
  (

In [128]:
med_std7 = std_by_med(a17_centers, b47_centers, 50, 60, 70, a17_files, b47_files, True, True)
med_std7

0.0027377703573719507

In [129]:
data7 = rl.average_residual_flux(a17_centers[0], b47_centers[0], 50, 60, 70, 
                                 a17_files, b47_files, red_files = True, rf_slope = True)
aper_std7 = np.std(data7['a1_b4_res_flux']/np.median(data7['a1_b4_res_flux']))
aper_std7

0.0018418554559806973

## Results

In [130]:
bkg_comparison = Table(names = ('Test', 'Median Subtraction', 'Annular Aperture'))
for x in range(1,8):
    med_std = globals()['med_std%s' % x]
    aper_std = globals()['aper_std%s' % x]
    bkg_comparison.add_row([x, med_std, aper_std])
bkg_comparison

Test,Median Subtraction,Annular Aperture
float64,float64,float64
1.0,0.00158008677614,0.00158376430412
2.0,0.00233282575477,0.00234329357954
3.0,0.00244624147973,0.00159605882287
4.0,0.00241792162148,0.00185892391054
5.0,0.0025451652979,0.00201751635688
6.0,0.00247720254818,0.00136553780825
7.0,0.00273777035737,0.00184185545598
